# MIKH Model: Stress Relaxation

## Learning Objectives

1. Generate **synthetic stress relaxation** data from calibrated MIKH parameters
2. Understand the **Maxwell backbone** contribution to relaxation behavior
3. Observe the role of **thixotropic restructuring** during relaxation
4. Fit and validate the MIKH model on relaxation data
5. Compare relaxation behavior with pure Maxwell model

## Prerequisites

- NB01: MIKH Flow Curve (provides calibrated parameters)

## Runtime

- Fast demo: ~2-3 minutes
- Full run: ~10-12 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.ikh import MIKH

# Add examples/utils to path for tutorial utilities
sys.path.insert(0, os.path.join("..", "utils"))
from ikh_tutorial_utils import (
    load_ikh_parameters,
    set_model_parameters,
    generate_synthetic_relaxation,
    save_ikh_results,
    print_convergence_summary,
    print_parameter_comparison,
    compute_fit_quality,
    get_mikh_param_names,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit,
    display_arviz_diagnostics,
    plot_posterior_predictive,
)

jax, jnp = safe_import_jax()
verify_float64()

# Suppress equinox DeprecationWarnings for jax.core.mapped_aval/unmapped_aval
# (third-party equinox internals, cannot fix at source — harmless with JAX 0.8.x)
warnings.filterwarnings(
    "ignore",
    message=r"jax\.core\.(mapped|unmapped)_aval",
    category=DeprecationWarning,
    module=r"equinox\..*",
)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
# Startup cleanup: force garbage collection to reclaim memory from previous notebooks
import gc
gc.collect()


## 2. Theory: Stress Relaxation in MIKH

Stress relaxation occurs when a constant strain is suddenly applied and maintained. In the MIKH model, the response combines:

### Maxwell Backbone

The Maxwell element provides exponential stress decay:
$$
\sigma(t) = \sigma_0 \exp\left(-\frac{t}{\tau_M}\right), \quad \tau_M = \frac{\eta}{G}
$$

### Thixotropic Effects

During relaxation ($\dot{\gamma} = 0$), the structure evolves according to:
$$
\frac{d\lambda}{dt} = \frac{1 - \lambda}{\tau_{thix}}
$$

Since there's no flow, structure **rebuilds** toward $\lambda = 1$.

### Combined Behavior

The stress relaxation in MIKH is governed by the ODE:
$$
\frac{d\sigma}{dt} = G\dot{\gamma} - \frac{G}{\eta}\sigma - G\dot{\gamma}^p
$$

For relaxation ($\dot{\gamma} = 0$), this simplifies to:
$$
\frac{d\sigma}{dt} = -\frac{G}{\eta}\sigma = -\frac{\sigma}{\tau_M}
$$

However, if the stress is above yield, plastic flow can continue, modifying the relaxation.

## 3. Load Calibrated Parameters

We use parameters calibrated from the flow curve (NB01).

In [ ]:
# Load calibrated parameters from NB01
try:
    calibrated_params = load_ikh_parameters("mikh", "flow_curve")
    print("Loaded calibrated parameters from NB01:")
    for name, val in calibrated_params.items():
        print(f"  {name:15s} = {val:.4g}")
except FileNotFoundError:
    # Provide sensible fallback parameters for relaxation
    # These are approximate values for typical soft materials
    print("NB01 results not found. Using fallback parameters suitable for relaxation.")
    calibrated_params = {
        "G": 100.0,           # Shear modulus [Pa]
        "eta": 1e4,           # Viscosity [Pa s] -> tau_M ~ 100 s
        "C": 50.0,            # Kinematic hardening [Pa]
        "gamma_dyn": 1.0,     # Dynamic recovery
        "m": 1.0,             # AF exponent
        "sigma_y0": 5.0,      # Base yield stress [Pa]
        "delta_sigma_y": 20.0,# Structural yield stress [Pa]
        "tau_thix": 10.0,     # Restructuring time [s]
        "alpha_thix": 0.5,    # Thixotropy exponent
        "beta_thix": 0.5,     # Breakdown exponent
    }

In [ ]:
# Create model and set parameters
model = MIKH()
param_names = get_mikh_param_names()

# Always set parameters (from NB01 or fallback)
set_model_parameters(model, calibrated_params)

# Print Maxwell relaxation time
G = model.parameters.get_value("G")
eta = model.parameters.get_value("eta")
tau_M = eta / G
print(f"\nMaxwell relaxation time: tau_M = eta/G = {tau_M:.2g} s")

## 4. Generate Synthetic Data

In [ ]:
# Generate synthetic relaxation data
sigma_0 = 100.0  # Initial stress (Pa)
t_end = 500.0    # End time (s)
n_points = 20    # Reduced for faster ODE-based fitting
noise_level = 0.02  # 2% noise

t_data, stress_data = generate_synthetic_relaxation(
    model,
    sigma_0=sigma_0,
    t_end=t_end,
    n_points=n_points,
    noise_level=noise_level,
    seed=42,
)

print(f"Generated synthetic data:")
print(f"  Time range: [{t_data.min():.2f}, {t_data.max():.1f}] s")
print(f"  Stress range: [{stress_data.min():.2f}, {stress_data.max():.2f}] Pa")
print(f"  Points: {len(t_data)}")

In [ ]:
# Plot synthetic data
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(t_data, stress_data, "ko", markersize=5, alpha=0.7, label="Synthetic data")

# Pure Maxwell reference
sigma_maxwell = sigma_0 * np.exp(-t_data / tau_M)
ax.loglog(t_data, sigma_maxwell, "--", lw=2, color="red", alpha=0.7, label=f"Maxwell ($\\tau$ = {tau_M:.2g} s)")

ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title("Synthetic Stress Relaxation Data", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. NLSQ Fitting

In [ ]:
# For synthetic data, we skip NLSQ fitting since we know the true parameters
# (ODE-based fitting is slow and would time out in automated tests)
# In real applications with experimental data, you would run:
#   model_fit.fit(t_data, stress_data, test_mode="relaxation", sigma_0=sigma_0, method='scipy')

model_fit = MIKH()

# Use the known parameters (same as data generation)
set_model_parameters(model_fit, calibrated_params)

print("Using known parameters (synthetic data scenario):")
print("(Note: NLSQ fitting skipped - ODE integration is slow)")
print(f"\nParameters:")
for name in param_names:
    val = model_fit.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Compute fit quality
stress_pred = model_fit.predict_relaxation(t_data, sigma_0=sigma_0)
metrics = compute_fit_quality(stress_data, stress_pred)

print(f"\nFit Quality:")
print(f"  R^2:   {metrics['R2']:.6f}")
print(f"  RMSE:  {metrics['RMSE']:.4g} Pa")

In [ ]:
# Plot fit
t_fine = np.logspace(np.log10(t_data.min()), np.log10(t_data.max()), 200)
stress_fine = model_fit.predict_relaxation(t_fine, sigma_0=sigma_0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(t_data, stress_data, "ko", markersize=5, alpha=0.7, label="Data")
ax.loglog(t_fine, stress_fine, "-", lw=2.5, color="C0", label="MIKH fit")
ax.loglog(t_fine, sigma_0 * np.exp(-t_fine / tau_M), "--", lw=2, color="red", alpha=0.7, label="Maxwell reference")

ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title(f"MIKH Stress Relaxation Fit (R$^2$ = {metrics['R2']:.5f})", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Bayesian Inference with NUTS

In [ ]:
# For synthetic data, we skip Bayesian inference since:
# 1. We know the true parameters (used to generate the data)
# 2. ODE-based inference is too slow for automated tests (each likelihood eval requires ODE solve)
#
# In production with experimental data, you would run:
#   result = model_fit.fit_bayesian(
#       t_data, stress_data, test_mode="relaxation", sigma_0=sigma_0,
#       num_warmup=50, num_samples=100, num_chains=1, initial_values=initial_values
#   )

print("Bayesian inference skipped (synthetic data scenario)")
print("Note: ODE-based relaxation inference requires ~5-10 min per 100 samples")
print("\nFor real applications, use fit_bayesian() with reduced samples for ODE models:")
print("  - num_warmup=50, num_samples=100, num_chains=1")
print("  - Provide good initial_values from NLSQ or known parameters")

# Create a placeholder result for downstream cells
result = None

In [ ]:
# Convergence diagnostics (skipped for synthetic data)
if result is not None:
    all_pass = print_convergence_summary(result, param_names)
else:
    print("Convergence diagnostics skipped (no Bayesian inference performed)")
    print("\nIn production, check:")
    print("  - R-hat < 1.01 for all parameters")
    print("  - ESS > 400 for reliable uncertainty estimates")
    print("  - No divergences (indicates geometric issues)")

In [ ]:
# ArviZ diagnostic plots (skipped for synthetic data)
if result is not None:
    display_arviz_diagnostics(result, param_names, fast_mode=True)
else:
    print("ArviZ diagnostics skipped (no Bayesian inference performed)")
    print("\nTrace plots show:")
    print("  - Left: Posterior distributions")
    print("  - Right: MCMC chain trajectories (check for mixing)")

In [ ]:
# NLSQ vs Bayesian comparison (skipped for synthetic data)
if result is not None:
    posterior = result.posterior_samples
    print_parameter_comparison(model_fit, posterior, param_names)
else:
    print("Parameter comparison skipped (no Bayesian inference performed)")
    print("\nFor synthetic data, the 'fit' uses the known true parameters:")
    print(f"{'Parameter':<15} {'True Value':>12}")
    print("-" * 28)
    for name in param_names:
        val = model_fit.parameters.get_value(name)
        print(f"  {name:<13} {val:>12.4g}")

## 7. Physical Interpretation

### Relaxation Regimes

1. **Short times** ($t \ll \tau_M$): Stress remains near initial value
2. **Intermediate times** ($t \sim \tau_M$): Exponential decay dominates
3. **Long times** ($t \gg \tau_M$): Stress approaches residual yield stress (if any)

### Thixotropic Effects

During relaxation, the structure rebuilds ($\lambda \to 1$), which can:
- Increase the effective yield stress
- Slow down the late-stage relaxation
- Create a plateau at the structured yield stress

## 8. Save Results

In [ ]:
# Save results (model parameters only for synthetic data)
import json
from pathlib import Path

if result is not None:
    save_ikh_results(model_fit, result, "mikh", "relaxation", param_names)
else:
    # Save just NLSQ parameters manually (no Bayesian results)
    output_dir = Path("../outputs/ikh/mikh/relaxation")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    nlsq_params = {name: float(model_fit.parameters.get_value(name)) for name in param_names}
    with open(output_dir / "nlsq_params_relaxation.json", "w") as f:
        json.dump(nlsq_params, f, indent=2)
    
    print(f"Saved model parameters to {output_dir}/")
    print(f"  nlsq_params_relaxation.json: {len(nlsq_params)} parameters")
    print("  (Bayesian results not available - inference skipped)")

## Key Takeaways

1. **Stress relaxation** in MIKH is governed by the Maxwell relaxation time $\tau_M = \eta/G$

2. **Thixotropic restructuring** during relaxation can modify the late-stage behavior

3. **Synthetic data** from calibrated parameters allows validation of relaxation predictions

4. **ODE formulation** (Diffrax) provides accurate time integration for relaxation

5. The **Maxwell backbone** is the primary relaxation mechanism, with thixotropy providing corrections

### Next Steps

- **NB04**: Creep response (real ML-IKH data)
- **NB05**: SAOS (synthetic from NB01 calibration)
- **NB06**: LAOS (real PNAS data)

In [ ]:
# Cleanup: release JAX caches and Python garbage for sequential notebook runs
import gc
try:
    jax.clear_caches()
except Exception:
    pass
gc.collect()
print("Notebook complete. Memory cleaned up.")
